In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import layers, models

# Load in the data

In [ ]:
streamflow_data = pd.read_csv("data/streamflow_data/Final_Processed_Station_Data_Watershed.csv")
streamflow_data.dropna(inplace=True)
streamflow_data.head(5)

In [ ]:
station_lats = streamflow_data.iloc[0][1:]
station_lons = streamflow_data.iloc[1][1:]
watersheds = streamflow_data.iloc[2][1:]

# drop the first 3 rows
streamflow_data = streamflow_data.drop([0, 1, 2])
# drop the first column
streamflow_data = streamflow_data.drop(columns=["name"])
streamflow_data = streamflow_data.astype(float)
streamflow_data.head(5)

In [ ]:
# load in the precipitation and temperature data
rainfall_data = np.load("data/weather_data/rainfall_data.npy").astype(float)
snowfall_data = np.load("data/weather_data/snowfall_data.npy").astype(float)
max_temp_data = np.load("data/weather_data/max_temp_data_interp.npy").astype(float)
min_temp_data = np.load("data/weather_data/min_temp_data_interp.npy").astype(float)

# Split the data into training, validation, and test sets

In [ ]:
# define the fractions
train_frac = 7/9
val_frac = 1/9
test_frac = 1/9

assert train_frac + val_frac + test_frac == 1

# define the indices for the train, validation, and test sets
train_idx = int(train_frac * len(streamflow_data))
val_idx = int((train_frac + val_frac) * len(streamflow_data))
n_train = train_idx
n_val = val_idx - train_idx
n_test = len(streamflow_data) - val_idx

assert n_train + n_val + n_test == len(streamflow_data)

# standardize the data based on the training set
rainfall_data = (rainfall_data - np.mean(rainfall_data[:train_idx])) / np.std(rainfall_data[:train_idx])
snowfall_data = (snowfall_data - np.mean(snowfall_data[:train_idx])) / np.std(snowfall_data[:train_idx])
max_temp_data = (max_temp_data - np.mean(max_temp_data[:train_idx])) / np.std(max_temp_data[:train_idx])
min_temp_data = (min_temp_data - np.mean(min_temp_data[:train_idx])) / np.std(min_temp_data[:train_idx])
streamflow_data = (streamflow_data - streamflow_data.iloc[:train_idx].mean()) / streamflow_data.iloc[:train_idx].std()
streamflow_data = streamflow_data.to_numpy()

In [ ]:
# define the time window
n_groups = 26
group_size = 7
time_window = n_groups * group_size
n_channels = 4
grid_shape = rainfall_data.shape[1:]
n_stations = streamflow_data.shape[1]

combine_multiple_groups = False
if combine_multiple_groups:
    n_groups = 28
    pts_per_group = 7
    group_sizes = [1,7,14,30]
    time_window = np.sum(group_sizes) * pts_per_group

# create the training, validation, and test sets
x_intermediate = np.empty(np.shape(rainfall_data) + (n_channels,), dtype='single')
for i, data in enumerate([rainfall_data, snowfall_data, max_temp_data, min_temp_data]):
    x_intermediate[:,:,:,i] = data

def gen_train():
    for i in range(n_train - time_window):
        xx = tf.convert_to_tensor(x_intermediate[i:i+time_window].reshape(n_groups, *grid_shape, n_channels, group_size).mean(axis=-1))
        yy = streamflow_data[time_window + i,:]
        yield (xx, yy)

def gen_val():
    for i in range(n_val):
        xx = tf.convert_to_tensor(x_intermediate[i+n_train-time_window:i+n_train].reshape(n_groups, *grid_shape, n_channels, group_size).mean(axis=-1))
        yy = streamflow_data[val_idx + i,:]
        yield (xx, yy)

def gen_test():
    for i in range(n_test):
        xx = tf.convert_to_tensor(x_intermediate[i+val_idx-time_window:i+val_idx].reshape(n_groups, *grid_shape, n_channels, group_size).mean(axis=-1))
        yy = streamflow_data[val_idx + i,:]
        yield (xx, yy)

# Create a model

In [ ]:
def create_model():
    # Input layer
    cnn_input = layers.Input(shape=(n_groups, 22, 37, n_channels), name="Weather_Data_Input")
    
    # TimeDistributed CNN
    cnn = layers.TimeDistributed(layers.Conv2D(8, (3, 3), activation='relu', padding='same'))(cnn_input)
    cnn = layers.TimeDistributed(layers.Conv2D(16, (3, 3), activation='relu', padding='same'))(cnn)
    cnn = layers.TimeDistributed(layers.MaxPooling2D((2, 2)))(cnn)
    cnn = layers.TimeDistributed(layers.Flatten())(cnn)  # Flatten the grid

    # LSTM for temporal patterns
    lstm = layers.LSTM(64, return_sequences=False, activation='tanh')(cnn)

    # Fully connected layers
    dense = layers.Dense(32, activation='relu')(lstm)
    dense = layers.Dropout(0.2)(dense)  # Dropout for regularization
    output = layers.Dense(n_stations, activation='linear', name="Streamflow_Output")(dense)

    # Model definition
    model = models.Model(inputs=cnn_input, outputs=output)
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

model = create_model()
model.summary()

# Train an ensemble of the best model

In [72]:
raise NotImplementedError("Best model not yet determined")

NotImplementedError: Best model not yet determined

In [ ]:
# create datasets and train an ensemble
batch_size = 8
ensemble_size = 10
learning_rate = 0.01

early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                             mode='min',
                                             verbose=1,
                                             patience=5,
                                             restore_best_weights=True)
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss',
                                                factor=0.1,
                                                patience=2,
                                                verbose=1,
                                                mode='min')

for i in range(ensemble_size):
    print(f"Training model {i+1}/{ensemble_size}")

    train_dataset = tf.data.Dataset.from_generator(
        gen_train,
        (tf.float16, tf.float16),
        (tf.TensorShape([time_window, *grid_shape, n_channels]), tf.TensorShape([n_stations]))
    ).shuffle(n_train).batch(batch_size)

    val_dataset = tf.data.Dataset.from_generator(
        gen_val,
        (tf.float16, tf.float16),
        (tf.TensorShape([time_window, *grid_shape, n_channels]), tf.TensorShape([n_stations]))
    ).shuffle(n_val).batch(batch_size)

    model = create_model()
    model.fit(
        train_dataset,
        epochs=40,
        validation_data=val_dataset,
        verbose=1,
        callbacks=[early_stopping, reduce_lr],
    )

    # save the model
    model.save(f"models/ensemble_model_{i}.keras")


In [ ]:
# evaluate the ensemble
ensemble = []
for i in range(4):
    model = tf.keras.models.load_model(f"models/ensemble_model_{i}.keras")
    ensemble.append(model)

val_dataset = tf.data.Dataset.from_generator(
    gen_val,
    (tf.float16, tf.float16),
    (tf.TensorShape([time_window, *grid_shape, n_channels]), tf.TensorShape([n_stations]))
).batch(batch_size)

ensemble_predictions = []
for model in ensemble:
    ensemble_predictions.append(model.predict(val_dataset))

ensemble_mean = np.mean(ensemble_predictions, axis=0)

# calculate the RMSE
val_data = np.zeros((n_val, n_stations))
for i, (xx, yy) in enumerate(gen_val()):
    val_data[i] = yy

In [ ]:
# plot the model predictions for the first 20 stations using subplots
n_plot = 20

predictions = ensemble_mean

fig, axs = plt.subplots(n_plot // 2, 2, figsize=(16, n_plot * 2))
for i in range(n_plot):
    mse = np.mean((val_data[:, i] - predictions[:, i]) ** 2)
    station = i
    ax = axs[i // 2, i % 2]
    ax.plot(np.array(ensemble_predictions).T[i], c='k', alpha=0.2)
    ax.plot(val_data[:, station], label='Actual')
    ax.plot(predictions[:, station], label='Predicted')
    ax.set_title(f"Station {station} (MSE: {mse:.3f})")
    ax.legend()
plt.tight_layout()
plt.show()